In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator,\
    DayLocator, MONDAY,date2num,num2date,AutoDateLocator
from matplotlib.finance import quotes_historical_yahoo_ohlc, candlestick_ohlc,candlestick2_ochl,volume_overlay3

from stockapp import models as stkmd
from dataapp import models as dtamd
from dataapp import tasks as dtatks
from dataapp import libs as dtalibs
from featureapp import libs as ftlibs
from featureapp import models as ftmd
from stockapp import tasks as stktks
from stockapp import libs as stklibs
import featureapp.models as ftmd
import featureapp.tasks as fttks
import queryapp.models as qrymd
import queryapp.tasks as qrytks

import featureapp as ftapp
import utility as uty
from utility import models as utymd
import itertools as itt
import multiprocessing as mp
from django.db import connection,connections
from django.db import reset_queries
import time 
import pandas as pd
import numpy as np
import inspect
import imp
import datetime
from talib.abstract import *
import utility.models as utmd
import stockapp.libs as stklib
from utility import codemanager as cdmng
from utility import maintenance as mnt

import json
from django.contrib.auth.models import AnonymousUser
import threading

stk=stkmd.Stockmeta.objects.get(Symbol='TSLA')
Fromdate=pd.datetime(2008,1,1)
Todate=pd.datetime.today()
Trange=pd.date_range(Fromdate,Todate)
Trange=[T.date() for T in Trange if T.weekday()<=4]

# fttks.computefeatuers(stk.id,Trange)

/usr/local/lib/python2.7/dist-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [3]:
featurecodes=ftmd.FeatureComputeCode.objects.all()
for computecode in featurecodes:
    computeclass=computecode.importcomputeclass()
    CC=computeclass(6264,Trange)
    CC.computeall(skipdone=True)
    CC.saveall()
#     del CC



TIMING : GetStockData time = 0.328475 on 2017-05-21 11:54:17 with args: ((6264,),{}) 
TIMING : addindicators time = 0.002141 on 2017-05-21 11:54:17 with args: ((            Close   Open   High    Low  Volume Symbol    SMA20
Date                                ,{}) 
TIMING : addindicators time = 0.002214 on 2017-05-21 11:54:17 with args: ((            Close   Open   High    Low  Volume Symbol    SMA20  SMAstd20
Date                      ,{}) 
TIMING : addindicators time = 0.002259 on 2017-05-21 11:54:17 with args: ((            Close   Open   High    Low  Volume Symbol    SMA20  SMAstd20  \
Date                   ,{}) 
TIMING : addindicators time = 0.002977 on 2017-05-21 11:54:17 with args: ((            Close   Open   High    Low  Volume Symbol    SMA20  SMAstd20  \
Date                   ,{}) 
TIMING : addindicators time = 0.002081 on 2017-05-21 11:54:17 with args: ((            Close   Open   High    Low  Volume Symbol    SMA20  SMAstd20  \
Date                   ,{}) 
TIMING : addin

In [4]:
print CC.df.shape
print CC.df.index[2]
CC.df['EMALowPoly2win4Fit'].iloc[2]

(3414, 24)
2003-10-17


{u'EMA8_max': 15.771658653846158,
 u'EMA8_mean': 15.58627606242527,
 u'EMA8_min': 15.35,
 u'EMAstd8_max': 0.2828427124746193,
 u'EMAstd8_mean': 0.25137579201042914,
 u'EMAstd8_min': 0.21546431367240648,
 u'err': [-0.75,
  -0.22500000000000142,
  0.10155440414507666,
  -0.02165865384615806],
 u'err_max': 0.10155440414507666,
 u'err_mean': -0.2237760624252707,
 u'err_min': -0.75,
 u'poly': [-0.16205326449780852, 0.7373176377540855, -0.76256609331407]}

In [5]:
Trange=[T.date() for T in pd.date_range(pd.datetime(2002,1,1),pd.datetime.today()) if T.weekday()<=4]
ftmd.FeaturesData.objects.filter(Symbol__id=6264,T__in=Trange).count()

3414

In [ ]:
Trange[-100:]

In [7]:
df=ftlibs.GetFeature(Symbolids=[6264])
print df.shape
print df.index[2]
df['EMALowPoly2win4Fit'].iloc[2]
# df.index

TIMING : GetFeature time = 0.683464 on 2017-05-21 11:56:10 with args: ((),{'Symbolids': [6264]}) 
(3414, 21)
2003-10-17 00:00:00


{u'EMA8_max': 15.771658653846158,
 u'EMA8_mean': 15.58627606242527,
 u'EMA8_min': 15.35,
 u'EMAstd8_max': 0.2828427124746193,
 u'EMAstd8_mean': 0.25137579201042914,
 u'EMAstd8_min': 0.21546431367240648,
 u'err': [-0.75,
  -0.22500000000000142,
  0.10155440414507666,
  -0.02165865384615806],
 u'err_max': 0.10155440414507666,
 u'err_mean': -0.2237760624252707,
 u'err_min': -0.75,
 u'poly': [-0.16205326449780852, 0.7373176377540855, -0.76256609331407]}

In [ ]:
df[['EMA8','EMAstd8']]

In [ ]:
df['FutPROFIT30days']


In [ ]:
CC.saveall()

In [ ]:
def replaceNaN2None(x):
    print x
    if type(x)==list:
        for i in range(len(x)):
            x[i]=replaceNaN2None(x[i])

        return x
    elif type(x)==tuple:
        a=[]
        for i in range(len(x)):
            a.append(replaceNaN2None(x[i]))
        x=tuple(a)
        return x

    elif type(x)==dict:
        for k,v in x.items():
            x[k]=replaceNaN2None(v)
        return x
    else:
        if pd.isnull(x):
            return None	
        else:
            return x

D={'a':[1,2,np.nan],'b':{'c':None,'d':['a',np.nan,'gg',None]}}
E=replaceNaN2None(D)
E

In [ ]:
import logging
logger=logging.getLogger('debug')
logger.info("timing")

In [ ]:
logger

In [ ]:
stk.id


# Running Features

In [ ]:
print "---------------- features-------------------"
featurecodes=ftmd.FeatureComputeCode.objects.all()
computecode=featurecodes[0]
computeclass=computecode.importcomputeclass()
CF=computeclass(stk.id,Trange)
CF.computeall(skipdone=True)
# CF.saveall()
print CF.getfeaturelist()

CF.df=CF.addindicators(CF.df,[
        {'name':'SMAstd','timeperiod':20,'colname':'SMAstd20'},
        {'name':'EMAstd','timeperiod':8,'colname':'EMAstd8'},
    ])

In [ ]:
print "---------------- Queries-------------------"
querycodes=qrymd.QueryComputeCode.objects.all()
computecode=querycodes[0]
computeclass=computecode.importcomputeclass()
CQ=computeclass(stk.id,Trange)
CQ.df=CF.df.copy()
CQ.computeall(skipdone=True)
# CQ.saveall()
print CQ.OutcomeCharts()

CQ.OutcomeCharts(perf=['FutPROFIT10days','FutLOSS10days','FutPROFIT30days','FutLOSS30days'])

# CQ.chartfeatures(addpricecols=(),addfeatcols=[
#     ['CCI5','CCI50'],
#     ['PastPROFIT10days','PastLOSS10days','PastPROFIT30days','PastLOSS30days'],
# #     [],
# #     ['FutPROFIT10days','FutLOSS10days']
# ],
#  addquerycols=[
#     'CCICHERRIES',
# ],
# ip=5562)

In [ ]:
# CQ.chartfeatures(addpricecols=(),addfeatcols=[
# #     ['CCI5','CCI50'],
#     ['PastPROFIT10days','PastLOSS10days','PastPROFIT30days','PastLOSS30days'],
# #     [],
# #     ['FutPROFIT10days','FutLOSS10days']
# ],
#  addquerycols=[
#     'CCICHERRIES','SMA20BOUNCE1','SMA20BOUNCE2','SMA20BOUNCE3'
# ],
# ip=5562)


CQ.chartfeatures(addpricecols=(),addfeatcols=[
#     ['CCI5','CCI50'],
    ['PastPROFIT10days','PastLOSS10days','PastPROFIT30days','PastLOSS30days'],
#     [],
#     ['FutPROFIT10days','FutLOSS10days']
],
 addquerycols=[
    'CHERRYSMABOUNCE',
],
ip=5562)

In [ ]:
import json
import simplejson
def applyrollingfunc(df,newcolname,func,window,edge='right'):
    df[newcolname]=np.nan
    for i in range(1,len(df)-1):
        if edge=='right':
            dw=df.loc[ df.index[i-window:i],:  ]
        elif edge=='left':
            dw=df.loc[ df.index[i:i+window],:  ]
        else:
            dw=df.loc[ df.index[max(i-int(window/2),0):i+int(window/2)],:  ]
        df.loc[df.index[i],newcolname]=func(dw)
        
    return df[newcolname]

def getsignal(dw):
    n=int( len(dw)/2.0 )
    err=(dw['Low']-dw['SMA20']).values
    p=np.polyfit(np.arange(len(err)),err,2)
    mid=dw.index[n]
#     print p,mid
    if p[0]>0 and abs(min(err))<=0.5*dw['SMAstd20'].mean():
        return 1
    else:
        return 0
def getsignal(dw):
    n=int( len(dw)/2.0 )
    err=(dw['Low']-dw['SMA20']).values
    p=np.polyfit(np.arange(len(err)),err,2)
    s=[]
    for a in p:
        if np.isnan(a):
            s.append(None)
        else:
            s.append(a)
    return json.dumps( {'poly':s,'SMAstd20_mean':dw['SMAstd20'].mean(),'SMAstd20_max':dw['SMAstd20'].max(),'SMAstd20_min':dw['SMAstd20'].min()})

def getbouncefeatures(dw):
    n=int( len(dw)/2.0 )
    err=(dw['Low']-dw['SMA20']).values
    p=np.polyfit(np.arange(len(err)),err,2)
    p=map(lambda x : None if np.isnan(x) else x,p)
    return json.dumps({'poly':p,
                        'err_mean':np.mean(err),'err_max':max(err),'err_min':min(err),
                        'SMAstd20_mean':dw['SMAstd20'].mean(),'SMAstd20_max':dw['SMAstd20'].max(),'SMAstd20_min':dw['SMAstd20'].min()
                        })

CF.df['SMALowPoly2win4Fit']=applyrollingfunc(CF.df,'SMALowPoly2win4Fit',getbouncefeatures,4,edge='center')

# CF.df=applyrollingfunc(CF.df,'mysignal',getsignal,4,edge='center')
# print CF.df.shape,CF.df[CF.df['mysignal']==1].shape
# CQ.chartfeatures(addpricecols=(),addfeatcols=[
# #     ['CCI5','CCI50'],
#     ['PastPROFIT10days','PastLOSS10days','PastPROFIT30days','PastLOSS30days'],
# #     [],
# #     ['FutPROFIT10days','FutLOSS10days']
# ],
#  addquerycols=[
#     'mysignal','CCICHERRIES'
# ],
# ip=5562)

In [ ]:
CF.df['SMALowPoly2win4Fit'].iloc[1]

# Running Queries

In [ ]:
querycodes=qrymd.QueryComputeCode.objects.all()
computecode=querycodes[0]
computeclass=computecode.importcomputeclass()
CQ=computeclass(stk.id,Trange)
CQ.computeall(skipdone=True)
CQ.saveall()

In [ ]:
CQ.OutcomeCharts()

In [ ]:
CQ.getquerylist()

In [ ]:
CQ.chartfeatures(addpricecols=(),ip=5562,
addfeatcols=[
    ['CCI5','CCI50'],
    ['PastPROFIT10days','PastLOSS10days'],['FutPROFIT10days','FutLOSS10days']
],
addquerycols=[
    'CCICHERRIES',
]
)

# Misc

In [ ]:
def applyrollingfunc(df,newcolname,func,window,edge='right'):
    df[newcolname]=np.nan
    for i in range(1,len(df)-1):
        if edge=='right':
            dw=df.loc[ df.index[i-window:i],:  ]
        elif edge=='left':
            dw=df.loc[ df.index[i:i+window],:  ]
        else:
            dw=df.loc[ df.index[max(i-int(window/2),0):i+int(window/2)],:  ]
        df.loc[df.index[i],newcolname]=func(dw)
        
    return df

def getsignal(dw):
    n=int( len(dw)/2.0 )
    err=(dw['Low']-dw['SMA20']).values
    p=np.polyfit(np.arange(len(err)),err,2)
    mid=dw.index[n]
    
    if len(err[err>0])>len(err)/2 and p[0]>=0 and min(err)<=0.3*dw['SMAstd20'].max():
        return 1
    else:
        return 0
    
CF.df=applyrollingfunc(CF.df,'mysignal',getsignal,10,edge='center')
print CF.df.shape,CF.df[CF.df['mysignal']==1].shape
CQ.chartfeatures(addpricecols=(),addfeatcols=[
    ['CCI5','CCI50'],
    ['PastPROFIT10days','PastLOSS10days','PastPROFIT30days','PastLOSS30days'],
#     [],
#     ['FutPROFIT10days','FutLOSS10days']
],
 addquerycols=[
    'CCICHERRIES','mysignal',
],
ip=5562)

In [ ]:
window=10
i=4
# CF.df.index[[1,2,3,4]]
range(max(i-int(window/2),0),i+int(window/2) )
CF.df.loc[ CF.df.index[max(i-int(window/2),0):i+int(window/2)],:  ]

# Add Index

# Building Query

In [ ]:
url='http://ec2-54-183-21-11.us-west-1.compute.amazonaws.com:8080/?symbols=TSLA,MSFT,AAPL&from=2012-01-01&to=2017-01-01'
import urllib2
response = urllib2.urlopen(url)
html = response.read()
D=json.loads(html)

In [ ]:
D['query']

In [ ]:
print len(D['result'])
# pd.DataFrame(D['result'])
df=pd.DataFrame()
for d in D['result']:
    df=pd.concat([df,pd.DataFrame(d['quotes'])])
df['date']=df['date'].apply(lambda x : pd.to_datetime(x))
df

In [ ]:
Todate=pd.datetime(2017,1,1).date()
Fromdate=pd.datetime(2014,8,1).date()
stk=stkmd.Stockmeta.objects.get(Symbol='AAPL')
df=dtalibs.GetStockData([stk.id],Fromdate,Todate,'concat')

# uses close prices (default)
df[['Open','High','Low','Close','Volume']]
inputs1 = {
    'open': df['Open'].values,
    'high': df['High'].values,
    'low': df['Low'].values,
    'close': df['Close'].values,
    'volume': df['Volume'].values
}
output = SMA(inputs1, timeperiod=25)
df['sma20']=df['Close'].rolling(window=20).mean()
df['smastd20']=df['Close'].rolling(window=20).std()
df['ema20']=df['Close'].ewm(span=20).mean()
df['emastd20']=df['Close'].ewm(span=20).std(bias=False)

df['ema8']=df['Close'].ewm(span=8).mean()
df['emastd8']=df['Close'].ewm(span=8).std(bias=False)


df['sma50']=df['Close'].rolling(window=50).mean()
df['smastd50']=df['Close'].rolling(window=50).std()
df['ema50']=df['Close'].ewm(span=50).mean()
df['emastd50']=df['Close'].ewm(span=50).std(bias=False)

df['cci5'] = CCI(inputs1, timeperiod=5)
df['cci50']= CCI(inputs1, timeperiod=50)
df['hasCherries']=(df['cci50']-df['cci5'])>180

def sequentialcluster(dp):
    dp['date']=dp.index.copy()
    dp.index=range(len(dp))
    C=[]
    c=[]
    for i in dp.index[1:]:
        if (dp.loc[i,'date']-dp.loc[i-1,'date']).days<=1:
                c.append(dp.loc[i-1,'date'])
        else:
            if len(c)==0:
                c=[dp.loc[i-1,'date']]
            C.append(c)
            c=[]
    
    C=[c[int(len(c)/2)] for c in C if len(c)>0]
    dC=dp[dp['date'].isin(C)].copy()
    dC.index=dC['date'].copy()
    dC=dC[dC["Close"]>=dC['sma20']].copy()
    
#     for ind in dC.index:
    return dC
        
# C=sequentialcluster(dbsma20)


dp=df[(df.index<Todate) & (df.index>Fromdate)].copy()

dbsma20=dp[abs(dp['Close']-dp['ema20'])/dp['ema20']<= 0.6*dp['emastd20']/dp['ema20']].copy()
# kmeans = KMeans(n_clusters=30, random_state=0).fit(dbsma20['date'].values.reshape(-1,1))
# dateclusters=[pd.to_datetime(f) for f in kmeans.cluster_centers_.reshape(1,-1)[0]]

dC=sequentialcluster(dbsma20)
idx=[]
print len(dC)
for i in range(len(dC)):
    for j in range(len(dp)):
        if dp.index[j]==dC.index[i]:
            break
            
    if (dp.loc[dp.index[j-2]:dp.index[j+3],'Close']-dp.loc[dp.index[j-2]:dp.index[j+3],'sma20']).min()>0:
        idx.append(i)
dC=dC.iloc[idx]

fig,ax=plt.subplots(1,1,figsize=(15,7))
ax.plot(dp.index,dp['Close'],linewidth=3,label='close')
ax.plot(dp.index,dp['sma20'],'--',label='sma20')
ax.plot(dp.index,dp['sma50'],'--',label='sma50')
# ax.plot(dp.index,dp['sma20']+1*dp['smastd20'],'k--',label='+std20')
# ax.plot(dp.index,dp['sma20']-1*dp['smastd20'],'k--',label='-std20')
dc=dp[(dp['hasCherries']==True) & (dp['cci50']>100)].copy()
ax.plot(dc.index,dc['Close'],'r',marker='o',linestyle='',label='cherry')
ax.plot(dC.index,dC['Close'],'k',marker='*',markersize=10,linestyle='',label='bounce')


ax.legend()
fig,ax=plt.subplots(1,1,figsize=(15,2))
ax.plot(dp.index,dp['Volume'])
plt.show()

In [ ]:
def convexity()

In [ ]:
Todate=pd.datetime(2017,5,1).date()
Fromdate=pd.datetime(2010,8,1).date()
stk=stkmd.Stockmeta.objects.get(Symbol='TSLA')
df=dtalibs.GetStockData([stk.id],Fromdate,Todate,'concat')

# uses close prices (default)
df[['Open','High','Low','Close','Volume']]
inputs1 = {
    'open': df['Open'].values,
    'high': df['High'].values,
    'low': df['Low'].values,
    'close': df['Close'].values,
    'volume': df['Volume'].values
}
output = SMA(inputs1, timeperiod=25)
df['sma20']=df['Close'].rolling(window=20).mean()
df['smastd20']=df['Close'].rolling(window=20).std()
df['ema20']=df['Close'].ewm(span=20).mean()
df['emastd20']=df['Close'].ewm(span=20).std(bias=False)

df['ema8']=df['Close'].ewm(span=8).mean()
df['emastd8']=df['Close'].ewm(span=8).std(bias=False)


df['sma50']=df['Close'].rolling(window=50).mean()
df['smastd50']=df['Close'].rolling(window=50).std()
df['ema50']=df['Close'].ewm(span=50).mean()
df['emastd50']=df['Close'].ewm(span=50).std(bias=False)

df['cci5'] = CCI(inputs1, timeperiod=5)
df['cci50']= CCI(inputs1, timeperiod=50)
df['hasCherries']=(df['cci50']-df['cci5'])>180
df['datenum']=date2num(df.index)

def feature1(df,label=None):
#     dp=df[(df['hasCherries']==True) & (df['cci50']>100)].copy()
#     dp=df[df['cci50']>50].copy()
    dp=df.copy()
    dp['date']=dp.index.copy()
    dp['bottom']=dp[['Open','Close']].min(axis=1)

    dbsma20=dp[abs(dp['Low']-dp['ema20'])/dp['ema20']<= 0.2*dp['emastd20']/dp['ema20']].copy()
#     dbsma20=dp[abs(dp['bottom']-dp['sma20'])/dp['sma20']<= 0.3*dp['smastd20']/dp['sma20']].copy()
    
#     dbsma20.index=range(len(dbsma20))
#     C=[]
#     c=[]
#     for i in dbsma20.index[1:]:
#         if (dbsma20.loc[i,'date']-dbsma20.loc[i-1,'date']).days<=1:
#                 c.append(dbsma20.loc[i-1,'date'])
#         else:
#             if len(c)==0:
#                 c=[dbsma20.loc[i-1,'date']]
#             C.append(c)
#             c=[]
    
#     C=[c[int(len(c)/2)] for c in C if len(c)>0]
#     dC=dbsma20[dbsma20['date'].isin(C)].copy()
    
    
    dC=dbsma20
    
    
    dC.index=dC['date'].copy()

    idx=[]
    print len(dC)
    for i in range(len(dC)):
        for j in range(len(df)):
            if df.index[j]==dC.index[i]:
                break
#         if (df.loc[df.index[j-3]:df.index[j],'Low']-df.loc[df.index[j-3]:df.index[j],'sma20']).min()>0:
#             idx.append(i)
#     dC=dC.iloc[idx]
    
    df['SMA20bounce']=False
    for ind in dC.index:
        df.loc[ind,'SMA20bounce']=True
        ix=np.argwhere(df.index==ind)[0,0]
#         if np.any(df.loc[df.index[ix-10:ix+10],'hasCherries'].values==True):
#             df.loc[ind,'SMA20bounce']=True
#         else:
#             df.loc[ind,'SMA20bounce']=False
            
    return df

def GetPerformance(df,label):
    df['3monProf']=np.nan
    df['6monProf']=np.nan
    df['3monLoss']=np.nan
    df['6monLoss']=np.nan
    for ind in df[df[label]==True].index:
        S=df.loc[ind:(ind+pd.DateOffset(90)).date(),'Close']-df.loc[ind,'Close']
        df.loc[ind,'3monProf']=100*S[S>0].max()/df.loc[ind,'Close']
        df.loc[ind,'3monLoss']=100*S[S<0].min()/df.loc[ind,'Close']
        
        S=df.loc[ind:(ind+pd.DateOffset(180)).date(),'Close']-df.loc[ind,'Close']
        df.loc[ind,'6monProf']=100*S[S>0].max()/df.loc[ind,'Close']
        df.loc[ind,'6monLoss']=100*S[S<0].min()/df.loc[ind,'Close']
    return df

def MakeChart(dF,T0,T,featcols):
    df=dF[T0:T].copy()
    autodate=AutoDateLocator()
    mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
    alldays = DayLocator()              # minor ticks on the days
    weekFormatter = DateFormatter('%Y %b %d')  # e.g., Jan 12
    dayFormatter = DateFormatter('%d')      # e.g., 12

    fig, ax = plt.subplots(figsize=(30,10))
    fig.subplots_adjust(bottom=0.2)
    ax.xaxis.set_major_locator(autodate)
    ax.xaxis.set_minor_locator(alldays)
    ax.xaxis.set_major_formatter(weekFormatter)
    #ax.xaxis.set_minor_formatter(dayFormatter)

    quotes=[tuple(x) for x in df[['datenum','Open','High','Low','Close']].to_records(index=False)]
    #plot_day_summary(ax, quotes, ticksize=3)
    # candlestick2_ochl(ax, df['Open'], df['Close'], df['High'], df['Low'], width=4, colorup='k', colordown='r', alpha=0.75)
    candlestick_ohlc(ax, quotes, width=0.6)
    ax.plot(df['datenum'],df['sma20'],'--',label='sma20')
    ax.plot(df['datenum'],df['sma50'],'--',label='sma50')
    # ax.plot(dp.index,dp['sma20']+1*dp['smastd20'],'k--',label='+std20')
    # ax.plot(dp.index,dp['sma20']-1*dp['smastd20'],'k--',label='-std20')
#     dc=dp[(dp['hasCherries']==True) & (dp['cci50']>100)].copy()
    for ff in  featcols:
        dp=df[df[ff]==True]
        ax.plot(dp['datenum'],dp['Close'],'r',marker='o',markersize=20,linestyle='',label=ff)
#     ax.plot(dC['datenum'],dC['Close'],'k',marker='*',markersize=10,linestyle='',label='bounce')
#     ax.arrow( dC['datenum'].iloc[0], dC['Close'].iloc[0]+1,0, 1, fc="k", ec="k",head_width=0.05, head_length=0.1 )

    ax.xaxis_date()
    ax.autoscale_view()
    plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')

    plt.show()
    # mpld3.show()
    # mpld3.display(fig)

    

df=feature1(df,label=None)
MakeChart(df,pd.datetime(2010,1,1).date(),pd.datetime(2011,1,1).date(),['SMA20bounce'])
MakeChart(df,pd.datetime(2011,1,1).date(),pd.datetime(2012,1,1).date(),['SMA20bounce'])
MakeChart(df,pd.datetime(2012,1,1).date(),pd.datetime(2013,1,1).date(),['SMA20bounce'])
MakeChart(df,pd.datetime(2013,1,1).date(),pd.datetime(2014,1,1).date(),['SMA20bounce'])
MakeChart(df,pd.datetime(2014,1,1).date(),pd.datetime(2015,1,1).date(),['SMA20bounce'])
MakeChart(df,pd.datetime(2015,1,1).date(),pd.datetime(2016,1,1).date(),['SMA20bounce'])
MakeChart(df,pd.datetime(2016,1,1).date(),pd.datetime(2017,5,1).date(),['SMA20bounce'])


df=GetPerformance(df,'SMA20bounce')
plt.figure()
df[['3monProf','3monLoss']].hist(figsize=(20,10),bins=20)
plt.show()
plt.figure()
df[['6monProf','6monLoss']].hist(figsize=(20,10),bins=20)
plt.show()

color = dict(boxes='DarkGreen', whiskers='DarkOrange',medians='DarkBlue', caps='Gray')
plt.figure()
df[['3monProf','3monLoss','6monProf','6monLoss']].plot.box(color=color, sym='r+')
plt.show()

In [ ]:
def MakeChart(df,pricecols,querycols,featcols):
    """
    pricecols are additional columns that have same range as close
    querycols are true/false snapped to close price
    featcols are additional that are plotted below
    
    For example:
    pricecols=[{'colname':'sma20','plotargs':('g',),'plotkwargs':{'label':'sma20',}},
          {'colname':'sma50','plotargs':('r',),'plotkwargs':{'label':'sma50',}}]
    querycols=[{'colname':'hasCherries','plotargs':('y',),'plotkwargs':{'label':'hasCherries','marker':'o','markersize':15,'linestyle':''}}]
    featcols=[ [{'colname':'cci5','plotargs':('r--',),'plotkwargs':{'label':'cci5',}}],
               [ {'colname':'cci50','plotargs':('g',),'plotkwargs':{'label':'cci50',}}]
             ]  
    """
#     df=dF[T0:T].copy()
#     dfperf=dF[T:(T+pd.DateOffset(360)).date()].copy()
    
    autodate=AutoDateLocator()
    mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
    alldays = DayLocator()              # minor ticks on the days
    weekFormatter = DateFormatter('%Y %b %d')  # e.g., Jan 12
    dayFormatter = DateFormatter('%d')      # e.g., 12
    
    Nsubplots=len(featcols)
    fig, ax = plt.subplots(1,2,figsize=(15,7+2*Nsubplots))
#     plt.figure(num=None, figsize=(30, 10), dpi=80, facecolor='w', edgecolor='k')
#     fig.subplots_adjust(bottom=0.2)
    ax1 = plt.subplot2grid((2+Nsubplots, 1), (0, 0), rowspan=2)
    axft=[]
    for i in range(Nsubplots):
        axft.append(plt.subplot2grid((2+Nsubplots, 1), (2+i, 0),sharex=ax1))
        
    ax1.xaxis.set_major_locator(autodate)
    ax1.xaxis.set_minor_locator(alldays)
    ax1.xaxis.set_major_formatter(weekFormatter)
    
    

    quotes=[tuple(x) for x in df[['datenum','Open','High','Low','Close','Volume']].to_records(index=False)]
    ret=candlestick_ohlc(ax1, quotes, width=0.6)
    
    for prcl in pricecols:
        ax1.plot(df['datenum'],df[prcl['colname'] ],*prcl['plotargs'],**prcl['plotkwargs'] )
    
    L=df['High'].max()-df['Low'].min()
    mm=df['Volume'].max()
    ax1.bar(df['datenum'],0.5*L*df['Volume']/mm,bottom=df['Low'].min()-1.5,color='y',alpha=0.5)
#     volume_overlay3(ax[0], quotes, colorup='k', colordown='r', width=4, alpha=1.0)
    
    for qcl in  querycols:
        dp=df[df[qcl['colname']]==True]
        ax1.plot(dp['datenum'],dp['Close'],*qcl['plotargs'],**qcl['plotkwargs'])
    
    ax1.set_xlim(quotes[0][0],quotes[-1][0])
    ax1.set_ylim(df['Low'].min()*0.9,df['High'].max()*1.1)
    ax1.xaxis_date()
    ax1.xaxis.tick_top()
    ax1.autoscale_view()
    ax1.legend()
    ax1.grid()
    
    ax12 = ax1.twinx()
    ax12.set_ylim(df['Low'].min()*0.9,df['High'].max()*1.1)
    ax12.grid()
    
    plt.setp(ax1.get_xticklabels(), rotation=45, horizontalalignment='left')
        
    for i in range(len(featcols)):
        autodate=AutoDateLocator()
        mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
        alldays = DayLocator()              # minor ticks on the days
        weekFormatter = DateFormatter('%Y %b %d')  # e.g., Jan 12
        dayFormatter = DateFormatter('%d')      # e.g., 12
        
        axft[i].xaxis.set_major_locator(autodate)
        axft[i].xaxis.set_minor_locator(alldays)
        axft[i].xaxis.set_major_formatter(weekFormatter)
        
        ftymax=-1000000
        ftymin=100000
        for ft in featcols[i]:
            axft[i].plot(df['datenum'],df[ ft['colname'] ],*ft['plotargs'],**ft['plotkwargs'])
            ftymin=min([ftymin,df[ft['colname']].min()])
            ftymax=max([ftymax,df[ft['colname']].max()])

#         axft[i].set_ylim(ftymin*0.8,ftymax*1.2)
        axft[i].set_xlim(quotes[0][0],quotes[-1][0])
        axft[i].autoscale_view()
        axft[i].legend()
        axft[i].grid()
        
        if i==len(featcols)-1:
            axft[i].xaxis_date()
            plt.setp(axft[i].get_xticklabels(), rotation=45, horizontalalignment='right')
        else:
            axft[i].get_xaxis().set_visible(False)
            
    plt.show()

    
pricecols=[{'colname':'sma20','plotargs':('g',),'plotkwargs':{'label':'sma20',}},
          {'colname':'sma50','plotargs':('r',),'plotkwargs':{'label':'sma50',}}]
querycols=[{'colname':'hasCherries','plotargs':('y',),'plotkwargs':{'label':'hasCherries','marker':'o','markersize':15,'linestyle':''}}]
featcols=[ [{'colname':'cci5','plotargs':('r--',),'plotkwargs':{'label':'cci5',}}],
           [ {'colname':'cci50','plotargs':('g',),'plotkwargs':{'label':'cci50',}}]
         ]  
T0=pd.datetime(2010,1,1).date()
TF=pd.datetime(2011,1,1).date()
MakeChart(df[T0:TF].copy(),pricecols,querycols,featcols)

In [ ]:
df.columns

In [ ]:
import pickle as pkl
import zlib
msg={'featcols':['A'],'df':df.round(decimals=3)}
msg2={'featcols':['A'],'df':pkl.dumps( df.round(decimals=3))   }
p=pkl.dumps(msg)
p2=pkl.dumps(msg2)
z=zlib.compress(p)
z2=zlib.compress(p2)
print len(p),len(z),len(p2),len(z2)

In [ ]:
window=360
T0=df.index[0]
TF=(df.index[0]+pd.DateOffset(window) ).date()

In [ ]:
import zmq
import pandas as pd
import numpy as np
import time
import zlib
import pickle as pkl

context = zmq.Context()

#  Socket to talk to server
print("Connecting to hello world server")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5557")

msg={'featcols':['SMA20bounce'],'df':df.round(decimals=3)}
p=pkl.dumps(msg)
z=zlib.compress(p)
print len(p),len(z)

socket.send(z)

#  Get the reply.
message = socket.recv()
print message
socket.close()


In [ ]:
import json
class registerfeature(object):
    registry=[]
    def __init__(self,para,returntype=None,cache=True):
        self.para=para
        self.usecache=cache
        self.cache={}
    def __call__(self,func):
        
        self.name=func.__name__
        self.doc=func.__doc__
        self.registry.append([self.name,self.doc])
        
        def func2(*args,**kwargs):
            key=json.dumps([args[1:],kwargs])
            if self.usecache:
                if key in self.cache:
                    print "return cached result"
                    return self.cache[key]
                else:
                    print "computing and saving result"
                    self.cache[key]=func(*args,**kwargs)
                    return self.cache[key]
            else:
                return func(*args,**kwargs)
        func2.isfeature=True
        
        return func2

class foo(object):

    @registerfeature(3,returntype=float)
    def SMA10(self):
        """
        sma10
        """
        return 32
    
    @classmethod
    def getfeatures(cls):
        print cls.__dict__.items()
        return [x for x, y in cls.__dict__.items() if hasattr(cls.__dict__[x],'isfeature')]
    def __getitem__(self,feat):
        return getattr(self,feat)()


foo.getfeatures()
    
    
f=foo()
f.getfeatures()
# print f.SMA10()
# print f.SMA10()
# print f['SMA10']


In [ ]:
dd=dtalibs.GetStockData(stk.id)
dd.sort_index()

In [ ]:
Fromdate=pd.datetime(2008,1,1)
Todate=pd.datetime.today()
Trange=pd.date_range(Fromdate,Todate)
Trange=[T.date() for T in Trange if T.weekday()<=4]

pd.DataFrame(list(ftmd.FeaturesData.objects.filter(T__in=Trange).values_list('Featuredata',flat=True) ))
# featdata.Featuredata

In [ ]:
D={}
D[(1,[1,2,3])]=4